### Importing Libraries

In [1]:
import tensorflow as tf
from tqdm import notebook
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import os

print(tf.__version__)
print(keras.__version__)

2.3.0
2.4.0


### Hyper Parameters

In [2]:
learning_rate = 0.001
training_epochs = 3
batch_size = 100

tf.random.set_seed(777)

### Creating a Checkpoint Directory

In [3]:
cur_dir = os.getcwd()
ckpt_dir_name = 'checkpoints'
model_dir_name = 'mnist_cnn_seq'

checkpoint_dir = os.path.join(cur_dir, ckpt_dir_name, model_dir_name)
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_prefix = os.path.join(checkpoint_dir, model_dir_name)

### MNIST/Fashion MNIST Data

In [4]:
# MNIST data
mnist = keras.datasets.mnist
class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

### Datasets

In [5]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# 정규화
train_images = train_images.astype(np.float32) / 255.
test_images = test_images.astype(np.float32) / 255.

# 채널 추가
train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)

# to categorical
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).\
                                                    shuffle(buffer_size=100000).\
                                                    batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).\
                                                   batch(batch_size)

### Model Function

In [14]:
def create_model():
    inputs = keras.Input(shape=(28,28,1))
    conv1 = keras.layers.Conv2D(filters=32, kernel_size=3,
                                activation=tf.nn.relu, padding='SAME')(inputs)
    pool1 = keras.layers.MaxPool2D(padding='SAME')(conv1)
    conv2 = keras.layers.Conv2D(filters=64, kernel_size=3,
                                activation=tf.nn.relu, padding='SAME')(pool1)
    pool2 = keras.layers.MaxPool2D(padding='SAME')(conv2)
    conv3 = keras.layers.Conv2D(filters=128, kernel_size=3,
                                activation=tf.nn.relu, padding='SAME')(pool2)
    pool3 = keras.layers.MaxPool2D(padding='SAME')(conv3)
    pool3_flat = keras.layers.Flatten()(pool3)
    dense4 = keras.layers.Dense(units=256, activation=tf.nn.relu)(pool3_flat)
    drop4 = keras.layers.Dropout(rate=0.4)(dense4)
    logits = keras.layers.Dense(units=10)(drop4)
    
    return keras.Model(inputs=inputs, outputs=logits)

In [15]:
model = create_model()
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 128)         73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 4, 4, 128)        

### Loss Function

In [16]:
@tf.function
def loss_func(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(
        y_pred=logits, y_true=labels, from_logits=True))
    return loss

### Calculating Gradient

In [17]:
@tf.function
def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_func(model, images, labels)
    return tape.gradient(loss, model.variables)

### Calculating Model's Accuracy

In [18]:
@tf.function
def evaluate(model, images, labels):
    logits = model(images, training=False) # training=True : dropout 사용 유무
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

### Optimizer

In [19]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

### Creating a Checkpoint

In [20]:
checkpoint = tf.train.Checkpoint(cnn=model)

### Training

In [21]:
@tf.function
def train(model, images, labels):
    grads = grad(model, images, labels)
    optimizer.apply_gradients(zip(grads, model.variables))

In [22]:
train_dataset

<BatchDataset shapes: ((None, 28, 28, 1), (None, 10)), types: (tf.float32, tf.float32)>

In [23]:
for epoch in range(training_epochs):
    avg_loss = 0.
    avg_train_acc = 0.
    avg_test_acc = 0.
    train_step = 0
    test_step = 0
    
    for images, labels in notebook.tqdm(train_dataset):
        train(model, images, labels)
        loss = loss_func(model, images, labels)
        acc = evaluate(model, images, labels)
        avg_loss += loss
        avg_train_acc += acc
        train_step += 1
    
    avg_loss /= train_step
    avg_train_acc /= train_step
    
    for images, labels in test_dataset:
        acc = evaluate(model, images, labels)
        avg_test_acc += acc
        test_step += 1
    
    avg_test_acc /= test_step
    
    print('Epoch:', '{}'.format(epoch + 1), 'loss =', '{:.8f}'.format(avg_loss), 
          'train accuracy = ', '{:.4f}'.format(avg_train_acc), 
          'test accuracy = ', '{:.4f}'.format(avg_test_acc))
    
    checkpoint.save(file_prefix=checkpoint_prefix)


Epoch: 1 loss = 0.16665772 train accuracy =  0.9570 test accuracy =  0.9813



Epoch: 2 loss = 0.04005614 train accuracy =  0.9901 test accuracy =  0.9918



Epoch: 3 loss = 0.02593659 train accuracy =  0.9934 test accuracy =  0.9909
